In [43]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import scipy.stats as ss
from scipy.stats import levene
import os
import pingouin as pg
import statsmodels.api as sm
import statsmodels.formula.api as smf
%matplotlib inline
os.getcwd()

'/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-berezin-33/Statistics'

In [44]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-i-berezin-33/Statistics/cars.csv', sep=',')

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

#### Сколько пропущенных значений встретилось в датасете?

In [46]:
df.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

#### Сколько всего уникальных марок машины встречается в датасете? 

In [47]:
df.head(2)

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0


In [48]:
df['company'] = df['CarName'].apply(lambda x: x.split(' ')[0])

In [49]:
df['company'].nunique()

28

In [50]:
rep = {'maxda' : 'mazda',
    'Nissan' : 'nissan',
    'porcshce' : 'porsche',
    'toyouta' : 'toyota',
    'vokswagen' : 'volkswagen',
    'vw' : 'volkswagen'}

In [51]:
df['company'] = df['company'].map(rep).fillna(df['company']) 

In [52]:
df['company'].nunique()

22

#### Чему равна корреляция между price и horsepower? Ответ округлите до 2 знаков после точки.

In [53]:
name_col = ['price', 'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'
]

In [54]:
df = df.drop(columns=['car_ID', 'CarName'])

In [55]:
df.columns

Index(['symboling', 'fueltype', 'aspiration', 'doornumber', 'carbody',
       'drivewheel', 'enginelocation', 'wheelbase', 'carlength', 'carwidth',
       'carheight', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize',
       'fuelsystem', 'boreratio', 'stroke', 'compressionratio', 'horsepower',
       'peakrpm', 'citympg', 'highwaympg', 'price', 'company'],
      dtype='object')

In [56]:
sample_df = df[name_col]
sample_df

,price,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower
0,13495.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
1,16500.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
2,16500.0,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154
3,13950.0,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102
4,17450.0,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,16845.0,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,2952,ohc,four,141,3.78,114
201,19045.0,volvo,gas,turbo,sedan,rwd,109.1,188.8,68.8,3049,ohc,four,141,3.78,160
202,21485.0,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,3012,ohcv,six,173,3.58,134
203,22470.0,volvo,diesel,turbo,sedan,rwd,109.1,188.8,68.9,3217,ohc,six,145,3.01,106


In [57]:
corr_mat = sample_df.corr()
corr_mat

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower
price,1.000000,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139
wheelbase,0.577816,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294
carlength,0.682920,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623
carwidth,0.759325,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732
curbweight,0.835305,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739
enginesize,0.874145,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769
boreratio,0.553173,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677
horsepower,0.808139,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000


In [58]:
corr_mat['price'].sort_values(ascending=False).round(2)

price         1.00
enginesize    0.87
curbweight    0.84
horsepower    0.81
carwidth      0.76
carlength     0.68
wheelbase     0.58
boreratio     0.55
Name: price, dtype: float64

##### Перевод в категориальные данные

In [59]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   price           205 non-null    float64
 1   company         205 non-null    object 
 2   fueltype        205 non-null    object 
 3   aspiration      205 non-null    object 
 4   carbody         205 non-null    object 
 5   drivewheel      205 non-null    object 
 6   wheelbase       205 non-null    float64
 7   carlength       205 non-null    float64
 8   carwidth        205 non-null    float64
 9   curbweight      205 non-null    int64  
 10  enginetype      205 non-null    object 
 11  cylindernumber  205 non-null    object 
 12  enginesize      205 non-null    int64  
 13  boreratio       205 non-null    float64
 14  horsepower      205 non-null    int64  
dtypes: float64(5), int64(3), object(7)
memory usage: 24.1+ KB


In [60]:
df_dummy = pd.get_dummies(data=sample_df[name_col], drop_first = True)
df_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 49 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   price                  205 non-null    float64
 1   wheelbase              205 non-null    float64
 2   carlength              205 non-null    float64
 3   carwidth               205 non-null    float64
 4   curbweight             205 non-null    int64  
 5   enginesize             205 non-null    int64  
 6   boreratio              205 non-null    float64
 7   horsepower             205 non-null    int64  
 8   company_audi           205 non-null    uint8  
 9   company_bmw            205 non-null    uint8  
 10  company_buick          205 non-null    uint8  
 11  company_chevrolet      205 non-null    uint8  
 12  company_dodge          205 non-null    uint8  
 13  company_honda          205 non-null    uint8  
 14  company_isuzu          205 non-null    uint8  
 15  compan

#### Построим небольшую модель всего с одним предиктором цены (price) – horsepower.

Какой процент изменчивости объясняет полученная модель? (округлите до целого)

In [61]:
result = smf.ols('price ~ horsepower', data=df_dummy).fit()

In [75]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Mon, 20 Mar 2023   Prob (F-statistic):           1.48e-48
Time:                        14:11:54   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

#### Построим две модели:
- Mодель со всеми предикторами
- Mодель со всеми предикторами, кроме марок машин

In [63]:
Y = sample_df['price']
X = df_dummy
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()

In [73]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.234e+29
Date:                Mon, 20 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:02:42   Log-Likelihood:                 4670.7
No. Observations:                 205   AIC:                            -9245.
Df Residuals:                     157   BIC:                            -9086.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  9.872e-

In [69]:
no_car_df = df_dummy.drop(columns=['company_audi', 'company_bmw', 'company_buick', 'company_chevrolet', 'company_dodge', 'company_honda', 'company_isuzu', 'company_jaguar', 'company_mazda', 'company_mercury', 'company_mitsubishi', 'company_nissan', 'company_peugeot', 'company_plymouth', 'company_porsche', 'company_renault', 'company_saab', 'company_subaru', 'company_toyota', 'company_volkswagen', 'company_volvo'])

In [70]:
# без машин
Y = sample_df['price']
X = no_car_df
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results_no_car = model.fit()

In [74]:
print(results_no_car.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.142e+30
Date:                Mon, 20 Mar 2023   Prob (F-statistic):               0.00
Time:                        14:02:57   Log-Likelihood:                 4768.8
No. Observations:                 205   AIC:                            -9482.
Df Residuals:                     177   BIC:                            -9389.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  4.119e-